.64

# Entrenando modelo BETO de detecta premisas y conclusiones en oraciones con PoC

# Introduction

# Setup

**Montar disco de Gdrive**

In [1]:
from google.colab import drive
drive.mount('/content/drive')
# usa drive de jesusmiguelgglang1@g


Mounted at /content/drive


In [2]:
!ls /content/drive/MyDrive/Exp_Objetivos2021/folders

fold_0_test.txt   fold_2_test.txt   fold_4_test.txt   fold_6_test.txt	fold_8_test.txt
fold_0_train.txt  fold_2_train.txt  fold_4_train.txt  fold_6_train.txt	fold_8_train.txt
fold_1_test.txt   fold_3_test.txt   fold_5_test.txt   fold_7_test.txt	fold_9_test.txt
fold_1_train.txt  fold_3_train.txt  fold_5_train.txt  fold_7_train.txt	fold_9_train.txt


Download `transformers` and install required packages.

In [3]:
#%%capture
!git clone https://github.com/huggingface/transformers
%cd transformers
!pip install .
!pip install -r ./examples/requirements.txt
%cd ..


Cloning into 'transformers'...
remote: Enumerating objects: 166111, done.
remote: Counting objects: 100% (279/279), done.
remote: Compressing objects: 100% (189/189), done.
remote: Total 166111 (delta 120), reused 162 (delta 79), pack-reused 165832
Receiving objects: 100% (166111/166111), 166.88 MiB | 25.09 MiB/s, done.
Resolving deltas: 100% (125294/125294), done.
/content/transformers
Processing /content/transformers
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 28.6 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-4.35.0.dev0-py3-none-any.whl size=7884083 sha256=7aff7974fc16b5

In [4]:
#usar info de huggigfaces
# https://github.com/huggingface/transformers/tree/master/examples/token-classification

# !wget -O 'requirements_tokenclasify.txt' "https://raw.githubusercontent.com/huggingface/transformers/master/examples/token-classification/requirements.txt"

!pip install -r "/content/drive/MyDrive/Exp_Objetivos2021/config_ner/requirements.txt"


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.1 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=ba5c8b50e6f94351f5bb087c840c90f6c3649c0d055074b613ee5311bc49c615
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


# Data


## 1. Download Datasets

Para deteccion de componentes

   - dev.txt: Spanish test data for the development stage
   - test.txt: Spanish test data
   - train.txt: Spanish train data

In [5]:
!ls /content/drive/MyDrive/Exp_Objetivos2021/folders

fold_0_test.txt   fold_2_test.txt   fold_4_test.txt   fold_6_test.txt	fold_8_test.txt
fold_0_train.txt  fold_2_train.txt  fold_4_train.txt  fold_6_train.txt	fold_8_train.txt
fold_1_test.txt   fold_3_test.txt   fold_5_test.txt   fold_7_test.txt	fold_9_test.txt
fold_1_train.txt  fold_3_train.txt  fold_5_train.txt  fold_7_train.txt	fold_9_train.txt


## 2. Preprocessing - quizas usar otro modelo bert-base-multilingual-cased

Let's define some variables that we need for further pre-processing steps and training the model:

In [6]:
MAX_LENGTH = 120 #@param {type: "integer"}
MODEL = "dccuchile/bert-base-spanish-wwm-uncased"  #@param ["chriskhanhtran/spanberta", "bert-base-multilingual-cased","dccuchile/bert-base-spanish-wwm-uncased", "pysentimiento/robertuito-base-uncased"]

# dccuchile/bert-base-spanish-wwm-uncased

The script below will split sentences longer than `MAX_LENGTH` (in terms of tokens) into small ones. Otherwise, long sentences will be truncated when tokenized, causing the loss of training data and some tokens in the test set not being predicted.

In [7]:
%%capture
# !wget "https://raw.githubusercontent.com/stefan-it/fine-tuned-berts-seq/master/scripts/preprocess.py"

!cp "/content/drive/MyDrive/Exp_Objetivos2021/config_ner/preprocess.py" "/content/"

In [8]:
!ls

drive  preprocess.py  sample_data  transformers


In [ ]:
# !python3 preprocess.py ./drive/MyDrive/experimentosArgComp/datos_compOracionesArgC_folds/fold_0_train.txt $MODEL $MAX_LENGTH > train.txt
# # !python3 preprocess.py ./drive/MyDrive/experimentosArgComp/datos_comp/fold_0_dev.txt $MODEL $MAX_LENGTH > dev.txt
# !python3 preprocess.py ./drive/MyDrive/experimentosArgComp/datos_compOracionesArgC_folds/fold_0_test.txt $MODEL $MAX_LENGTH > test.txt

## 3. Labels

In comp, there are have X classes of NER tags:

- O, Outside of a component
- B-Arg, Beginning of arg
- I-Arg, arg entity


If your dataset has different labels or more labels than CoNLL-2002/2003 datasets, run the line below to get unique labels from your data and save them into `labels.txt`. This file will be used when we start fine-tuning our model.

In [ ]:
# !cat train.txt  test.txt | cut -d " " -f 2 | grep -v "^$"| sort | uniq > labels.txt
# !cat train.txt dev.txt test.txt | cut -d " " -f 2 | grep -v "^$"| sort | uniq > labels.txt

In [ ]:
# !head -n20 labels.txt

In [9]:
!ls

drive  preprocess.py  sample_data  transformers


# Fine-tuning Model


These are the example scripts from `transformers`'s repo that we will use to fine-tune our model for NER. After 04/21/2020, Hugging Face has updated their example scripts to use a new `Trainer` class. To avoid any future conflict, let's use the version before they made these updates.

In [10]:
!pip install conllu

In [11]:
%%capture
# !wget "https://raw.githubusercontent.com/huggingface/transformers/master/examples/legacy/token-classification/run_ner.py"
# !wget "https://raw.githubusercontent.com/huggingface/transformers/master/examples/legacy/token-classification/utils_ner.py"
# !wget "https://raw.githubusercontent.com/huggingface/transformers/master/examples/legacy/token-classification/tasks.py"


!cp "/content/drive/MyDrive/Exp_Objetivos2021/config_ner/run_ner.py" "/content/"
!cp "/content/drive/MyDrive/Exp_Objetivos2021/config_ner/tasks.py" "/content/"
!cp "/content/drive/MyDrive/Exp_Objetivos2021/config_ner/utils_ner.py" "/content/"



In [12]:
!ls

drive  preprocess.py  run_ner.py  sample_data  tasks.py  transformers  utils_ner.py


In [13]:
!ls /content/drive/MyDrive/Exp_Objetivos2021/folders

fold_0_test.txt   fold_2_test.txt   fold_4_test.txt   fold_6_test.txt	fold_8_test.txt
fold_0_train.txt  fold_2_train.txt  fold_4_train.txt  fold_6_train.txt	fold_8_train.txt
fold_1_test.txt   fold_3_test.txt   fold_5_test.txt   fold_7_test.txt	fold_9_test.txt
fold_1_train.txt  fold_3_train.txt  fold_5_train.txt  fold_7_train.txt	fold_9_train.txt


In [14]:
# grid search
 # Model training
max_accuracy = 0

# epoch 4
epoch = [160]   # 20 , 80,  160,  200     - 2, 3, 5, 10, 30, 60, 100, 200
# batch sizes: 8, 16, 32, 64, 128
# learning rates: 3e-4, 1e-4, 5e-5, 3e-5

# learning_rate choices
# learning_rates = [ 0.1, 0.2, 0.3, 0.4, 0.5, 0.01, 0.02, 0.03, 0.04, 0.05 ]
learning_rates = [3e-5] #  3e-6

# iterations choices
# iterations = [ 100, 200, 300, 400, 500 ]
batch = [16] # 16,32

# #folds
# folds = list(range(10))

# available combination of learning_rate and iterations

parameters = []
for z  in epoch:
    for j in batch:
      for i in learning_rates:
        parameters.append( ( i, j, z) )

print("Available combinations : ",len(parameters),  parameters )


# Applying linear searching in list of available combination
# to achieved maximum accuracy on CV set


Available combinations :  1 [(3e-05, 16, 160)]


In [15]:
  dir = "/content/drive/MyDrive/Exp_Objetivos2021/results/"

  # /content/drive/MyDrive/experimentosArgComp/results/22abrCorpus5arg

# /content/drive/MyDrive/experimentosArgComp/sample_data

# /content/drive/MyDrive/experimentosArgComp/results/BETOcorpus4compPCsolo

  #
  f = open(dir+"log_exp.txt", "a+")
  log_text = f"results\n"
  f.write(log_text)
  f.close()

In [16]:
pip install TensorRT

  Preparing metadata (setup.py) ... done
  Created wheel for TensorRT: filename=tensorrt-8.6.1.post1-py2.py3-none-any.whl size=17281 sha256=a8a0b41970b150179e49523c8435a998d5659eac6bb48fc540901dd6e3fbaa51
  Stored in directory: /root/.cache/pip/wheels/f4/c8/0e/b79b08e45752491b9acfdbd69e8a609e8b2ed7640dda5a3e59
Successfully built TensorRT


In [17]:
pip install --upgrade accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 4.2 MB/s eta 0:00:00


In [18]:
def read_examples_from_file(file_path):
    """Read words and labels from a CoNLL-2002/2003 data file.

    Args:
      file_path (str): path to NER data file.

    Returns:
      examples (dict): a dictionary with two keys: `words` (list of lists)
        holding words in each sequence, and `labels` (list of lists) holding
        corresponding labels.
    """
    with open(file_path, encoding="utf-8") as f:
        examples = {"words": [], "labels": []}
        words = []
        labels = []
        for line in f:
            if line.startswith("-DOCSTART-") or line == "" or line == "\n":
                if words:
                    examples["words"].append(words)
                    examples["labels"].append(labels)
                    words = []
                    labels = []
            else:
                splits = line.split(" ")
                words.append(splits[0])
                if len(splits) > 1:
                    labels.append(splits[-1].replace("\n", ""))
                else:
                    # Examples could have no label for mode = "test"
                    labels.append("O")
    return examples

In [ ]:
import time
from transformers import AutoTokenizer, AutoModelForTokenClassification
#model = TFAutoModelForSequenceClassification.from_pretrained("bert-base-cased")
# for i in range(100):
#   x = 100 + 11999 + 2000
#   current_time = time.time()
#   elapsed_time = current_time - start_time



# (0.0003, 8) learn y batch
#epoch = 10
corpus = "ArgPC"
max_test_f1 = 0
max_test_accuracy_score = 0
max_parametros = ""


for k in range( len( parameters ) ) :

  sumaF1 = 0
  sumaAcc = 0


  for fold in range(0,10):

    start_time = time.time()

    !rm train.txt
    !rm test.txt


    !python3 preprocess.py /content/drive/MyDrive/Exp_Objetivos2021/folders/fold_{fold}_train.txt $MODEL $MAX_LENGTH > train.txt
    !python3 preprocess.py /content/drive/MyDrive/Exp_Objetivos2021/folders/fold_{fold}_test.txt $MODEL $MAX_LENGTH > test.txt


    !cat train.txt  test.txt | cut -d " " -f 2 | grep -v "^$"| sort | uniq > labels.txt

    #epoch = parameters[k][2]
    epoch=160



    MAX_LENGTH = 128
    MODEL = "dccuchile/bert-base-spanish-wwm-uncased"
    #MODEL = "chriskhanhtran/spanberta"
    OUTPUT_DIR = "bert-base-ml-ner"
    BATCH_SIZE = parameters[k][1] #32
    NUM_EPOCHS = epoch
    SAVE_STEPS = 100
    LOGGING_STEPS = 100
    SEED = 42
    LEARNING_RATE = parameters[k][0]


    parametros = f"epoch_{str(epoch)}_learn_{str(LEARNING_RATE)}_batch_{str(BATCH_SIZE)}_corpus{corpus}_fold{str(fold)}\n"


    !python3 run_ner.py --data_dir ./ \
    --labels ./labels.txt \
    --model_name_or_path $MODEL \
    --output_dir $OUTPUT_DIR \
    --max_seq_length  $MAX_LENGTH \
    --num_train_epochs $NUM_EPOCHS \
    --per_gpu_train_batch_size $BATCH_SIZE \
    --save_steps $SAVE_STEPS \
    --seed $SEED \
    --do_train \
    --do_predict \
    --overwrite_cache \
    --overwrite_output_dir \
    --save_total_limit 2 \
    --learning_rate $LEARNING_RATE


    !cp ./bert-base-ml-ner/config.json  {dir}config_{parametros}.json
    !cp ./bert-base-ml-ner/test_predictions.txt   {dir}test_predictions_{parametros}.txt
    !cp ./bert-base-ml-ner/test_results.txt   {dir}test_results_{parametros}.txt

    archivo_result = "./bert-base-ml-ner/test_results.txt"
    test_f1_current = 0
    test_accuracy_score_current = 0

    current_time = time.time()
    elapsed_time = current_time - start_time

    testsite_array = []
    with open(archivo_result) as my_file:
        for line in my_file:
            testsite_array.append(line.split() )
            if line.split()[0] == "test_f1":
              print("test_f1",line.split()[2],parametros)
              test_f1_current = float(line.split()[2])
            if line.split()[0] ==  "test_accuracy_score":
              print("test_accuracy_score",line.split()[2],parametros)
              test_accuracy_score_current = float(line.split()[2])

    f = open(dir+"log_exp.txt", "a+")
    log_text = f"{str(test_accuracy_score_current),str(test_f1_current)},{str(epoch)},{str(LEARNING_RATE)},{str(BATCH_SIZE)},{corpus},{str(fold)},time,{str(elapsed_time)}\n"
    f.write(log_text)
    f.close()

    # guardo para acumular F1 y acc

    sumaAcc += test_accuracy_score_current
    sumaF1 += test_f1_current


    if max_test_f1 < test_f1_current :
      max_test_f1 = test_f1_current
      max_parametros = parametros

    #**********resultados
    import numpy as np
    from sklearn.metrics import classification_report
    !rm test.txt

    !python3 preprocess.py /content/drive/MyDrive/Exp_Objetivos2021/folders/fold_{fold}_test.txt $MODEL $MAX_LENGTH > test.txt
    !python3 preprocess.py /content/bert-base-ml-ner/test_predictions.txt $MODEL $MAX_LENGTH > prediction.txt

    !cat test.txt | cut -d " " -f 2 | grep -v "^$"| sort | uniq > labels.txt


    y_true = read_examples_from_file("test.txt")["labels"]
    y_pred = read_examples_from_file("prediction.txt")["labels"]
    print ("RESULTADOS FOLDER", fold)
    print(classification_report(np.concatenate(y_true), np.concatenate(y_pred)))

    #***********resultados


  print(max_test_f1,max_parametros)








rm: cannot remove 'train.txt': No such file or directory
rm: cannot remove 'test.txt': No such file or directory
2023-10-31 02:03:15.742769: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-31 02:03:15.742816: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-31 02:03:15.742858: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-31 02:03:16.843755: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
10/31/2023 02:03:18 - WARNING - __main__ -   Process rank: 0, device: cuda:0, n_gpu: 1, distributed training: True, 16-bits t